<a href="https://colab.research.google.com/github/KevinLolochum/BERT-MODELS/blob/main/DistilBERT_Extractive_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DistilBERT for Extractive Summarization using PyTorch Solution**

***1. Installing text summarizer, neuralcoref and spacy.***


* Spacy models enable corefencing explained [here](https://spacy.io/universe/project/neuralcoref). 
* Coreferencing is the ability of models to predict the antecedent that is being refered to later in the sentence/paragraph through ranking. 
* I will not use coferencing for this model but it is good to know and might use in future models.



In [ ]:
! pip install bert-extractive-summarizer
! pip install neuralcoref
! pip install spacy

In [ ]:
# Libraries

import spacy
import numpy as np
import pandas as pd
import torch
from transformers import DistilBertConfig, DistilBertModel, DistilBertTokenizer
from summarizer import Summarizer
from summarizer.coreference_handler import CoreferenceHandler

import en_core_web_sm


***2. Instantiating the Model***



In [ ]:
# Initiating model and tokenizer

Config = DistilBertConfig.from_pretrained('distilbert-base-cased')
Config.output_hidden_states = True
MAIN_MODEL = DistilBertModel.from_pretrained('bert-base-cased', config = Config)
TOKENIZER = DistilBertTokenizer.from_pretrained('bert-base-cased')

# Model

model = Summarizer(custom_model= MAIN_MODEL, custom_tokenizer= TOKENIZER)

***3. Loading COVID19 News Data from the Web and Cleaning***

* Ontario covid19 [news](https://news.ontario.ca/en/release/59437/ontario-expanding-innovative-home-and-community-care-services) and summary.



In [ ]:
body = ''' The Ontario government is investing more than $115 million to support 850 patients across the province with high, complex care needs access services in the comfort of their own homes, while also expanding virtual care options for patients across the province. This funding is part of the province's COVID-19 fall preparedness plan, Keeping Ontarians Safe: Preparing for Future Waves of COVID-19, which will enable the province to respond to future surges and waves of the virus, while preserving hospital capacity.

Details were provided today by Premier Doug Ford and Christine Elliott, Deputy Premier and Minister of Health.

"With today's modern medical advances and technology, and with the help of our dedicated nurses and personal support workers, we are ready to provide quality care for patients in their own homes and in their communities," said Premier Ford. "This kind of care is not only critical to keeping patients safe and healthy during COVID-19, but also part of our government's ongoing commitment to end hallway health care and ensure our hospitals are not overwhelmed."

The new High Intensity Supports at Home program helps patients with high care needs transition from a hospital back to their home or community setting with the right supports, also known as alternate level of care patients. This program provides integrated, team-based care where multiple types of services are wrapped around the patient. This can include up to 11 hours a day of personal support services and nursing services per patient. Other community services may include Meals on Wheels, homemaking and transportation to medical appointments.

With this investment, the province will add more capacity in hospitals, provideapproximately 484,000 nursing and therapy visits, and approximately 1.4 million personal support worker hours.

"Our government is making it easier and faster for patients to access high-quality care at home," said Minister Elliott. "Connecting patients with the care they need, when and where they need it will help keep Ontarians healthy and avoid unnecessary visits and stays at the hospital. This program will help ensure Ontario's hospitals have the capacity needed to address those impacted by the COVID-19 pandemic."

As part of Ontario's continued response to the COVID-19 outbreak and in alignment with the Digital First for Health Strategy, Ontario is investing $14.5 million to support the expansion of virtual care for Ontario Health Teams and frontline home and community care service providers. Ontario has seen a significant increase in the number of visits provided virtually across Ontario. This investment will equip health care providers with the support needed to sustain current capacity and improve the quality of care delivered virtually. Virtual care in home and community care settings will also benefit rural and remote area regions where access to care is more challenging.

An investment of $9.5 million will go towards remote patient monitoring programs delivered by various health care organizations, including Ontario Health Team member organizations like hospitals. This will support the delivery of remote patient care, monitoring and symptom management at home to support COVID-19 patients and other vulnerable populations. The implementation of these remote patient monitoring programs will allow patients to connect virtually with a health care provider for routine checkups, symptom monitoring, referral to other providers, and escalation where necessary to a virtual or in-person medical assessment.

As part of this funding, $5 million will be invested in virtual home and community care capacity by providing funding directly to frontline home and community care service providers. This investment will help purchase technologies for video conferencing, remote monitoring and secure messaging that will enable care to continue to be delivered at home, minimizing face-to-face visits where appropriate, and keeping clients and families out of the hospital.

These investments will offer more choices in how patients receive care and make health care simpler, easier and more convenient, while allowing patients to continue accessing the routine health care services they need throughout the pandemic.

The government continues to build a better, more connected healthcare system through the creation of Ontario Health Teams, which are bringing together health care providers to work as one team to improve patient outcomes. Through an Ontario Health Team, patients will experience easier transitions from one provider to another, including, for example, between hospitals and home care providers, with one patient story, one patient record and one care plan. Patients and families will also have access to 24/7 navigation and care coordination services. Forty-two Ontario Health Teams have been approved to date.'''

In [ ]:
Human_summary = ''' The Ministry of Health and Ontario Health are supporting a needs-based application process for reviewing virtual care investment proposals submitted through the Ontario Health Regional lead by home and community care service providers and Ontario Health Teams.
Proposals will be selected based on their capacity and readiness, and the services will be available in early 2021.
Virtually delivered home and community care is provided where it is appropriate to the needs and preferences of the patient and their family. 
In-person home and community care has been shown to be safe and continues to be the primary way that most people receive these services.
Virtual care is intended to complement in-person care as appropriate and will not replace in-person visits. 
In-person care will continue to be offered for patients who prefer it and when it is clinically necessary.
During the COVID-19 outbreak, around 7,500 virtual home care nursing and rehab visits have been delivered per week.
Two thirds of Canadians are interested in consulting with various health care providers through a virtual platform.
For additional protection against COVID-19, the Ontario government is encouraging everyone to download the COVID Alert app on their smartphone from the Apple and Google Play app stores. '''

***4. Creating the summary and evaluation***


* Now we can pass the corpus above through our model to create the summary and evaluate
* There are a number of parameters that you can specify e.g size of output summary relative to input corpus (ratio) and minimum length per sentence(min_length).





In [ ]:
# Exploring the output
result = model(body, ratio=0.25, min_length=25)
summary = ''.join(result)
print(summary)

The Ontario government is investing more than $115 million to support 850 patients across the province with high, complex care needs access services in the comfort of their own homes, while also expanding virtual care options for patients across the province. This kind of care is not only critical to keeping patients safe and healthy during COVID-19, but also part of our government's ongoing commitment to end hallway health care and ensure our hospitals are not overwhelmed." This can include up to 11 hours a day of personal support services and nursing services per patient. Ontario has seen a significant increase in the number of visits provided virtually across Ontario. This investment will equip health care providers with the support needed to sustain current capacity and improve the quality of care delivered virtually. This will support the delivery of remote patient care, monitoring and symptom management at home to support COVID-19 patients and other vulnerable populations. Forty-

Calculating accuracy score using ROGUE.

In [ ]:
! pip install rouge/requirements.txt
! pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer('rouge1', use_stemmer=True)
scores = scorer.score(Human_summary, summary)